In [10]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import *
from tqdm import tqdm
from numba import njit
import random
from sklearn.metrics.pairwise import cosine_similarity

plt.rcParams["figure.figsize"] = [14, 8]
plt.rcParams["figure.dpi"] = 100
plt.set_cmap("binary_r")

<Figure size 1400x800 with 0 Axes>

# Boids model
Consider a flock of birds. If they are located in a too dense area, then they want to repell each other. 
At some intermediate distance, for example in a distance where they can see. each other, then birds will get attracted to each other and move in the same direction. In the same time, the flock will also try to cluster to the center of mass. The behvaior can be summarized as:
1. Separation
2. Alignment 
3. Cohesion

In [37]:
#np.random.seed(42)

# Simulation parameters
r1, r2, r3 = 2, 3, 5
N = 100  # Number of particles
x1, x2 = -10, 10
v1, v2 = -1, 1
dt = 1e-2
rho1, rho2, rho3, rho4 = [0.04, 0.01, 0.05, 0.9]
eta = 2 * np.pi / 3  # random factor upper bound (in radians)

# Animation constants 
time_steps = 10000
factor = 50
frames = time_steps // factor

position_history= np.zeros([frames, N, 2])
xy = np.random.uniform(x1, x2, [N, 2])  # Particle positions
v  = np.random.uniform(v1, v2, [N, 2])  # Particle velocities

# Function for computing distances
def get_distances(xy):
    separation = xy[:, None] - xy[None, :]
    pair_wise_distance = np.linalg.norm(separation, axis=-1)
    return pair_wise_distance

# Function for computing theta1
def theta1(xy):
    center_of_mass = np.mean(xy, axis=0) 
    repulsion_vectors = xy - center_of_mass
    theta = np.arctan2(repulsion_vectors[:, 1], repulsion_vectors[:, 0])
    return theta

# Function for computing theta2
def theta2(v):
    mean_velocity = np.mean(v, axis=0)
    theta = np.arctan2(mean_velocity[1], mean_velocity[0])
    return theta

# Function for computing theta3
def theta3(xy):
    center_of_mass = np.mean(xy, axis=0) 
    attraction_vectors = center_of_mass - xy
    theta = np.arctan2(attraction_vectors[:, 1], attraction_vectors[:, 0])
    return theta


f = 0
for t in tqdm(range(time_steps)):
    if t % factor == 0:
        position_history[f] = xy
        f += 1
        
    distance = get_distances(xy)

    repelling_indices  = np.logical_and(distance != 0, distance < r1)
    alignment_indices  = np.logical_and(distance >= r1, distance < r2)
    attracting_indices = np.logical_and(distance >= r2, distance < r3)

    theta1_values = theta1(xy)
    theta2_value = theta2(v)
    theta3_values = theta3(xy)

    # Update velocities
    for i in range(N):
        repelling_angles = theta1_values[repelling_indices[i]]
        attracting_angles = theta3_values[attracting_indices[i]]

        angle = (
            rho1 * np.mean(repelling_angles) if repelling_angles.size > 0 else 0
            + rho2 * theta2_value
            + rho3 * np.mean(attracting_angles) if attracting_angles.size > 0 else 0
            + rho4 * np.random.uniform(-eta, eta)
        )

        v[i] = [np.cos(angle), np.sin(angle)]

    # Update positions
    xy += v * dt

100%|██████████| 10000/10000 [00:22<00:00, 451.21it/s]


In [ ]:
def plot_animation(simulation_data):
    fig, ax = plt.subplots(figsize=(10, 10), dpi=100)

    def update(frame):
        ax.clear()
        ax.set_xlim(3*x1, 10*x2)
        ax.set_ylim(2*x1, 2*x2)
        ax.scatter(simulation_data[frame, :, 0], simulation_data[frame, :, 1])
        ax.set_title('Boids Simulation')

    anim = FuncAnimation(fig, update, frames=len(simulation_data), repeat=False)
    anim.save("boids_simulation.gif", writer="pillow")

plot_animation(position_history)